In [11]:
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma
from valentine.algorithms import Cupid
import os
import multiprocessing as mp
from multiprocessing import Pool
import time
import pandas as pd
from compute_match import *

In [2]:
path = "../json_datasets"

data_frames = {}
for file in os.listdir(path):
    filepath = f"{path}/{file}"
    data_frames[file] = (pd.read_json(filepath))

In [3]:
keys = list(data_frames.keys())
tuples = []
dfs_len = len(keys)
for i in range(dfs_len):
    for j in range(i + 1, dfs_len):
        dfl = data_frames[keys[i]]
        dfr = data_frames[keys[j]]
        tuples.append((keys[i], keys[j], dfl, dfr))

In [4]:
start_time = time.perf_counter()
with Pool() as pool:
    result = pool.map(calculate_match_coma_schema, tuples)
print("Program finished!")
finish_time = time.perf_counter()

Program finished!


In [8]:
columns = ["table_1", "table_2", "column_table_1", "column_table_2", "match_value"]
matches = pd.DataFrame(columns=columns)
print(result)
for res in result:
    for key in res[2].keys():
        d = dict()
        d["table_1"] = res[0]
        d["table_2"] = res[1]
        d["column_table_1"] = key[0][1]
        d["column_table_2"] = key[1][1]
        d["match_value"] = res[2][key]
        matches = pd.concat([matches, pd.DataFrame(d, index=[0])], ignore_index=True)
print(matches)

[('Silvestri-value.today.json', 'Gren-disfold.com.json', {}), ('Silvestri-value.today.json', 'Wissel-ariregister.rik.ee.json', {}), ('Silvestri-value.today.json', 'MalPatSaj-disfold.com.json', {}), ('Silvestri-value.today.json', 'DDD-ft.com.json', {}), ('Gren-disfold.com.json', 'Wissel-ariregister.rik.ee.json', {}), ('Gren-disfold.com.json', 'MalPatSaj-disfold.com.json', {(('table_1', 'industry'), ('table_2', 'Industry')): 0.9352676, (('table_1', 'country'), ('table_2', 'Country')): 0.9333069, (('table_1', 'sector'), ('table_2', 'Sector')): 0.931101, (('table_1', 'stock'), ('table_2', 'Stock')): 0.9279007, (('table_1', 'name'), ('table_2', 'Name')): 0.9257439, (('table_1', 'market_capitalization_USD'), ('table_2', 'MarketCap')): 0.4048583}), ('Gren-disfold.com.json', 'DDD-ft.com.json', {}), ('Wissel-ariregister.rik.ee.json', 'MalPatSaj-disfold.com.json', {}), ('Wissel-ariregister.rik.ee.json', 'DDD-ft.com.json', {}), ('MalPatSaj-disfold.com.json', 'DDD-ft.com.json', {})]
              

In [6]:
table_to_matches = dict()
for table in matches['table_1']:
    if table_to_matches.get(table) == None:
        table_to_matches[table] = 0
    table_to_matches[table] += 1
print(table_to_matches)

{'Gren-disfold.com.json': 6}


In [12]:
ground_truth = [('name', 'Name'), ('Name', 'Name')]
metrics = valentine_metrics.all_metrics(result, ground_truth)

print(metrics)

AttributeError: 'list' object has no attribute 'values'

In [7]:
pd.DataFrame(matches).to_csv("./instance_matches.csv")